# 概要
Pandasのtipsを見つけ次第まとめる

In [1]:
import numpy as np
import pandas as pd

## loc, ilocの確認
※ ixを使うのはもうやめよう
- locは列ラベルまたは行ラベルを指定する
- ilocは列番号または行番号を指定する
- iloc[A:B,A:B]のとき、A <= x < Bの番号が取得されることに注意する

In [10]:
# テストデータ
df_number = pd.DataFrame(np.arange(16).reshape(4,4), columns = ['A0', 'A1', 'A2', 'A3'])
df_number

,A0,A1,A2,A3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [16]:
# locは列ラベルまたは行ラベルを指定する
# df_number.loc[1,1] 列に"1"というラベルはないのでエラーとなる
df_number.loc[1, 'A1']

5

In [18]:
df_number.loc[:, 'A1' : 'A3'] # 連続していれば「:」でまたいで指定可能

,A1,A2,A3
0,1,2,3
1,5,6,7
2,9,10,11
3,13,14,15


In [19]:
# ilocは列番号または行番号を指定する
df_number.iloc[1, 1]

5

In [23]:
df_number.iloc[1:3, 1:4] # iloc[A:B,A:B]のとき、A <= x < Bの番号が取得されることに注意する

,A1,A2,A3
1,5,6,7
2,9,10,11


## CSVファイルの読み込み
pandasとdaskの読み込み方法
daskの方が大容量データの読み込みに向いている  
テストデータのirisは「/home/jovyan/work/01_pandas_cheetsheet/data/iris.csv」にあるとする

In [37]:
# ディレクトリの確認
import os
cwd = os.getcwd()
print(cwd)
print(os.listdir(cwd+'/data'))
iris_path = cwd+'/data/iris.csv'

/home/jovyan/work/01_pandas_cheetsheet
['iris.csv']


In [41]:
# pandasでの読み込み
df_iris = pd.read_csv(iris_path, sep=',', header=0) # ヘッダーを省略する場合はheader=Noneとする
df_iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [42]:
# daskでの読み込み
import dask.dataframe as dd
df_iris_dask = dd.read_csv(iris_path).compute()
df_iris_dask.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
